### Copyright 2024 Lehi Gracia

Ratings from analysts where the amount of analysts is greater than 50. Some stocks only have one or two and they are 100%

In [33]:
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime, timedelta
import pandas as pd
from pprint import pprint
from pymongo import MongoClient
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance

client = MongoClient('mongodb://localhost:27017/')
db = client['bigdata']
collection = db.daily_analysts_ratings

yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
today = datetime.today().strftime('%Y-%m-%d')


res = collection.find(
                 {'DATE':today, 'TOTAL':{"$gt":40}}, 
                 {'_id':0, 'script':0, 'timestamp':0}
                 )
data = pd.DataFrame(res).round(2)
print(data)


          DATE  TICK  UP  DOWN  HOLD    AVG  TOTAL
0   2025-01-07  AMZN  63     1     3  94.03     67
1   2025-01-07   WMT  38     1     3  90.48     42
2   2025-01-07  DDOG  40     0     6  86.96     46
3   2025-01-07  MSFT  52     0     5  91.23     57
4   2025-01-07  META  58     2     8  85.29     68
5   2025-01-07  NVDA  59     0     4  93.65     63
6   2025-01-07  AVGO  38     0     5  88.37     43
7   2025-01-07  AMZN  63     1     3  94.03     67
8   2025-01-07   WMT  38     1     3  90.48     42
9   2025-01-07  DDOG  40     0     6  86.96     46
10  2025-01-07  MSFT  52     0     5  91.23     57
11  2025-01-07  META  58     2     8  85.29     68
12  2025-01-07  NVDA  59     0     4  93.65     63
13  2025-01-07  AVGO  38     0     5  88.37     43


Daily extreme prices. 52 week low and high

In [35]:
collection_price = db.daily_extremes_52_week_price
collection_rsi = db.daily_extremes_52_week_rsi

db.daily_extremes_52_week_price.aggregate([
    {'$lookup': { 
        'from': db.daily_extremes_52_week_rsi,
        'localField': 'TICK',
        'foreignField': 'TICK',
        "pipeline": [
            { '$match': {
              '$expr': { '$eq': ['$$TICK', '$TICK']},
              '$expr': { '$eq': ['$$DATE', '$DATE']}
            }}
        ],
        'as': 'extremePRSI'
    }}
])

res = collection_price.extremePRSI({'DATE':yesterday}, 
                  {'_id':0, 'script':0, 'timestamp':0}
                  )

data = pd.DataFrame(res).round(2)
print(data)

InvalidDocument: cannot encode object: Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'bigdata'), 'daily_extremes_52_week_rsi'), of type: <class 'pymongo.synchronous.collection.Collection'>